In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## For Sunday

#### Drop columns with NA
#### Check out data from 2017 to 2021 and check demographics
#### Check police budgets by year or by borough if possible
#### See if you can calculate monthly crime rate
#### See if you can analyze crime, arrest, and crime rate forecasting

### Links
https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u/data
https://ibo.nyc.ny.us/RevenueSpending/nypd.html
https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78
https://compstat.nypdonline.org/2e5c3f4b-85c1-4635-83c6-22b27fe7c75c/view/89
https://compstat.nypdonline.org/2e5c3f4b-85c1-4635-83c6-22b27fe7c75c/view/89

In [2]:
data = pd.read_csv("NYPD_Complaint_Data_2017-21.csv")
data.head(5)

C:\Users\david\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,216044876,01/01/2017,11:00:00,08/20/2018,12:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,...,M,NaN,40.762334,-73.826917,"(40.76233421800004, -73.82691730799998)",PATROL BORO QUEENS NORTH,NaN,<18,ASIAN / PACIFIC ISLANDER,F
1,112025807,01/01/2017,09:00:00,09/01/2019,09:00:00,13.0,12/06/2019,233,SEX CRIMES,175.0,...,M,NaN,40.736698,-73.983316,"(40.736698070000045, -73.98331602599995)",PATROL BORO MAN SOUTH,NaN,45-64,WHITE HISPANIC,M
2,669902330,01/01/2017,00:01:00,12/31/2017,23:59:00,114.0,11/26/2019,233,SEX CRIMES,175.0,...,M,NaN,40.769306,-73.915508,"(40.76930608700008, -73.91550817999997)",PATROL BORO QUEENS NORTH,NaN,<18,BLACK,F
3,579530319,01/01/2017,00:00:00,12/01/2019,00:00:00,71.0,11/30/2019,112,THEFT-FRAUD,739.0,...,NaN,NaN,40.665023,-73.957114,"(40.66502269100005, -73.95711436799998)",PATROL BORO BKLYN SOUTH,NaN,25-44,BLACK,M
4,232211816,01/01/2017,00:01:00,12/31/2017,23:00:00,61.0,11/26/2019,112,THEFT-FRAUD,739.0,...,NaN,NaN,40.607011,-73.964718,"(40.607011439000075, -73.96471823499998)",PATROL BORO BKLYN SOUTH,NaN,25-44,ASIAN / PACIFIC ISLANDER,M


In [3]:
data = data.drop(['CMPLNT_TO_DT','CMPLNT_TO_TM', 'LOC_OF_OCCUR_DESC','TRANSIT_DISTRICT','PARKS_NM','HADEVELOPT', 
                  'HOUSING_PSA', 'STATION_NAME'], axis=1)

In [4]:
data.isna().sum()

CMPLNT_NUM                0
CMPLNT_FR_DT              0
CMPLNT_FR_TM              0
ADDR_PCT_CD               3
RPT_DT                    0
KY_CD                     0
OFNS_DESC                48
PD_CD                  1316
PD_DESC                1316
CRM_ATPT_CPTD_CD          0
LAW_CAT_CD                0
BORO_NM                1352
PREM_TYP_DESC          6696
JURIS_DESC                0
JURISDICTION_CODE      1316
X_COORD_CD               72
Y_COORD_CD               72
SUSP_AGE_GROUP       441017
SUSP_RACE            441017
SUSP_SEX             441017
Latitude                 72
Longitude                72
Lat_Lon                  72
PATROL_BORO            1316
VIC_AGE_GROUP             5
VIC_RACE                  5
VIC_SEX                   5
dtype: int64

### Begin calculating Monthly crime rate and check census data for 2017-2018 data
### Begin finding demographic data and processing spatial data for base regression
#### Use 5 year ACS


In [6]:
data['CMPLNT_FR_DT'].dtype

dtype('O')

In [7]:
data.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,216044876,01/01/2017,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,25-44,ASIAN / PACIFIC ISLANDER,M,40.762334,-73.826917,"(40.76233421800004, -73.82691730799998)",PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F
1,112025807,01/01/2017,09:00:00,13.0,12/06/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,UNKNOWN,UNKNOWN,M,40.736698,-73.983316,"(40.736698070000045, -73.98331602599995)",PATROL BORO MAN SOUTH,45-64,WHITE HISPANIC,M
2,669902330,01/01/2017,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,45-64,BLACK,M,40.769306,-73.915508,"(40.76930608700008, -73.91550817999997)",PATROL BORO QUEENS NORTH,<18,BLACK,F
3,579530319,01/01/2017,00:00:00,71.0,11/30/2019,112,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,...,NaN,NaN,NaN,40.665023,-73.957114,"(40.66502269100005, -73.95711436799998)",PATROL BORO BKLYN SOUTH,25-44,BLACK,M
4,232211816,01/01/2017,00:01:00,61.0,11/26/2019,112,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,...,NaN,NaN,NaN,40.607011,-73.964718,"(40.607011439000075, -73.96471823499998)",PATROL BORO BKLYN SOUTH,25-44,ASIAN / PACIFIC ISLANDER,M


In [8]:
len(data)

1792148

### Steps to calculate crime rate

- population of nyc in that year? or each borough?
- calculate crime rate for each month per 100k people

In [7]:
### 2020 & 2021 population based on 2020 census data, 2017-2019 is based on 1 yr ACS survey for respective year
bor = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten"]
year = [2017, 2018, 2019, 2020, 2021]
manhattan = [1664727, 1628701, 1628706, 1694251, 1694251]
brooklyn = [2648771, 2582830, 2559903, 2736074, 2736074]
queens = [2358582, 2278906, 2253858, 2405464, 2405464]
bronx = [1471160, 1432132, 1408207, 1472654, 1472654]
staten = [479458, 476179, 476143, 495747]

In [8]:
## General NYC population by year (2017-2021)
nyc = [8622698, 8398748, 8336817, 8804190, 8804190]

In [9]:
data["PATROL_BORO"].unique()

array(['PATROL BORO QUEENS NORTH', 'PATROL BORO MAN SOUTH',
       'PATROL BORO BKLYN SOUTH', 'PATROL BORO BRONX',
       'PATROL BORO QUEENS SOUTH', 'PATROL BORO BKLYN NORTH',
       'PATROL BORO STATEN ISLAND', 'PATROL BORO MAN NORTH', nan],
      dtype=object)

In [10]:
borough = []
for i in range(len(data["PATROL_BORO"])):
    if data["PATROL_BORO"][i]=='PATROL BORO QUEENS NORTH' or data["PATROL_BORO"][i]=='PATROL BORO QUEENS SOUTH':
        borough.append('queens')
    elif data["PATROL_BORO"][i]=='PATROL BORO BKLYN SOUTH' or data["PATROL_BORO"][i]=='PATROL BORO BKLYN NORTH':
        borough.append('brooklyn')
    elif data["PATROL_BORO"][i]=='PATROL BORO MAN SOUTH' or data["PATROL_BORO"][i]=='PATROL BORO MAN NORTH':
        borough.append('manhattan')
    elif data["PATROL_BORO"][i]=='PATROL BORO BRONX':
        borough.append('bronx')
    elif data["PATROL_BORO"][i]=='PATROL BORO STATEN ISLAND':
        borough.append('staten')
    else:
        borough.append('NA')
data["borough"] = borough

In [41]:
len(borough)

1792148

## Look at how to make a single data format to make loop for years to then create column with populations

In [11]:
data['CMPLNT_FR_DT'] = data['CMPLNT_FR_DT'].astype(str)

In [12]:
data['CMPLNT_FR_DT'] = pd.to_datetime(data['CMPLNT_FR_DT'].astype(str), format='%m/%d/%Y') 

In [13]:
data['date'] = pd.to_datetime(data['CMPLNT_FR_DT'],format='%m%Y', errors='coerce').dt.to_period('m')

In [14]:
data['year'] = data['date'].dt.strftime('%Y')
data['date'] = data['date'].dt.strftime('%Y%m')

In [15]:
data.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,Latitude,Longitude,Lat_Lon,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,borough,date,year
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,40.762334,-73.826917,"(40.76233421800004, -73.82691730799998)",PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017
1,112025807,2017-01-01,09:00:00,13.0,12/06/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,40.736698,-73.983316,"(40.736698070000045, -73.98331602599995)",PATROL BORO MAN SOUTH,45-64,WHITE HISPANIC,M,manhattan,201701,2017
2,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,40.769306,-73.915508,"(40.76930608700008, -73.91550817999997)",PATROL BORO QUEENS NORTH,<18,BLACK,F,queens,201701,2017


In [16]:
crime_num = pd.DataFrame(data['date'].value_counts())
crime_num = crime_num.rename(columns={"date": "crimes"})
crime_num = crime_num.reset_index()
#crime_num.head()

In [18]:
data = pd.merge(left=data, right=crime_num, left_on='date', right_on='index')


In [19]:
#data =data.drop(['index_x', 'index_y', 'crimes_y'], axis=1)
data.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,Lat_Lon,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,borough,date,year,index,crimes
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,"(40.76233421800004, -73.82691730799998)",PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,201701,38645
1,112025807,2017-01-01,09:00:00,13.0,12/06/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,"(40.736698070000045, -73.98331602599995)",PATROL BORO MAN SOUTH,45-64,WHITE HISPANIC,M,manhattan,201701,2017,201701,38645
2,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,"(40.76930608700008, -73.91550817999997)",PATROL BORO QUEENS NORTH,<18,BLACK,F,queens,201701,2017,201701,38645


In [20]:
'NA' in data['borough']

False

In [21]:
data['location_yr'] = data['borough']+data['year']
data.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,borough,date,year,index,crimes,location_yr
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,201701,38645,queens2017
1,112025807,2017-01-01,09:00:00,13.0,12/06/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO MAN SOUTH,45-64,WHITE HISPANIC,M,manhattan,201701,2017,201701,38645,manhattan2017
2,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO QUEENS NORTH,<18,BLACK,F,queens,201701,2017,201701,38645,queens2017


In [22]:
population = pd.DataFrame(data['location_yr'].unique())
pop = [queens[0], manhattan[0], brooklyn[0], bronx[0], staten[0], 'na',
      queens[1], manhattan[1], brooklyn[1], bronx[1], staten[1], 'na',
      queens[2], manhattan[2], brooklyn[2], bronx[2], staten[2], 'na',
      queens[3], manhattan[3], brooklyn[3], bronx[3], staten[3], 'na']
population['pop'] = pop
data = pd.merge(left=data, right=population, left_on='location_yr', right_on=0)
data.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,VIC_RACE,VIC_SEX,borough,date,year,index,crimes,location_yr,0,pop
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,201701,38645,queens2017,queens2017,2358582
1,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,BLACK,F,queens,201701,2017,201701,38645,queens2017,queens2017,2358582
2,336910010,2017-01-01,12:00:00,104.0,11/14/2019,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,...,WHITE,F,queens,201701,2017,201701,38645,queens2017,queens2017,2358582
3,452130663,2017-01-01,12:00:00,113.0,11/21/2019,104,RAPE,155.0,RAPE 2,COMPLETED,...,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,201701,38645,queens2017,queens2017,2358582
4,785888415,2017-01-01,09:00:00,110.0,11/13/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,WHITE HISPANIC,F,queens,201701,2017,201701,38645,queens2017,queens2017,2358582


## Drop duplicate columns

In [62]:
#data = data.drop(['index', 0], axis=1)
data

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,borough,date,year,crimes,location_yr,pop
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,38645,queens2017,2358582
1,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO QUEENS NORTH,<18,BLACK,F,queens,201701,2017,38645,queens2017,2358582
2,336910010,2017-01-01,12:00:00,104.0,11/14/2019,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,...,PATROL BORO QUEENS NORTH,18-24,WHITE,F,queens,201701,2017,38645,queens2017,2358582
3,452130663,2017-01-01,12:00:00,113.0,11/21/2019,104,RAPE,155.0,RAPE 2,COMPLETED,...,PATROL BORO QUEENS SOUTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,38645,queens2017,2358582
4,785888415,2017-01-01,09:00:00,110.0,11/13/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO QUEENS NORTH,<18,WHITE HISPANIC,F,queens,201701,2017,38645,queens2017,2358582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792143,442023925,2020-12-28,20:35:00,49.0,12/28/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,45-64,WHITE HISPANIC,M,NA,202012,2020,30632,NA2020,na
1792144,936557394,2020-12-29,10:45:00,50.0,12/29/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na
1792145,494869817,2020-12-29,20:48:00,52.0,12/29/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na
1792146,719051633,2020-12-30,11:04:00,79.0,12/30/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na


## Calculate crime rate

In [67]:
#data['crimes'] = pd.to_numeric(data['crimes'])
#data['pop'] = pd.to_numeric(data['pop'])
df = data[data.pop != 'na']
df

KeyError: True

In [64]:
data

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,borough,date,year,crimes,location_yr,pop
0,216044876,2017-01-01,11:00:00,109.0,12/31/2019,116,SEX CRIMES,179.0,AGGRAVATED SEXUAL ASBUSE,COMPLETED,...,PATROL BORO QUEENS NORTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,38645,queens2017,2358582
1,669902330,2017-01-01,00:01:00,114.0,11/26/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO QUEENS NORTH,<18,BLACK,F,queens,201701,2017,38645,queens2017,2358582
2,336910010,2017-01-01,12:00:00,104.0,11/14/2019,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,...,PATROL BORO QUEENS NORTH,18-24,WHITE,F,queens,201701,2017,38645,queens2017,2358582
3,452130663,2017-01-01,12:00:00,113.0,11/21/2019,104,RAPE,155.0,RAPE 2,COMPLETED,...,PATROL BORO QUEENS SOUTH,<18,ASIAN / PACIFIC ISLANDER,F,queens,201701,2017,38645,queens2017,2358582
4,785888415,2017-01-01,09:00:00,110.0,11/13/2019,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,...,PATROL BORO QUEENS NORTH,<18,WHITE HISPANIC,F,queens,201701,2017,38645,queens2017,2358582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792143,442023925,2020-12-28,20:35:00,49.0,12/28/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,45-64,WHITE HISPANIC,M,NA,202012,2020,30632,NA2020,na
1792144,936557394,2020-12-29,10:45:00,50.0,12/29/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na
1792145,494869817,2020-12-29,20:48:00,52.0,12/29/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na
1792146,719051633,2020-12-30,11:04:00,79.0,12/30/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,25-44,BLACK,M,NA,202012,2020,30632,NA2020,na


## Run initial regression
## see if you can group crimes
## find additional demographic data for other models (add to "population" dataframe) to join to larger table
## Convert location to regression format


## Notes

- There is a significant number of observations where information regarding the suspect is not known. Instead of droppping, we can perhaps use this as part of the analysis, naming these observations as unknown and creating an indicator. It might reveal information about which crimes more often have less info about the suspect or if certain locations struggle with reporting full details of crime